# Data Import From MINIO server

## A. Accessing MINIO Console

Step 1: Head to this link http://10.1.32.31:9001 on your browser

Step 2: Login using your username and password

## B. Create test bucket in your name and upload few files

<img style="width: 60%" src="im/cap1.PNG">

<img style="width: 60%"  src='im/cap2.PNG'>

## C. Download data from MINIO to workspace

### 1. Import libraries

In [1]:
from minio import Minio
from dotenv import load_dotenv # add this line
import os

In [2]:
os.getcwd()

'/workspace/best_practices_1'

### 2. Create .env file with credentials 

#### Run these in the terminal

`echo "MINIO_ACCESS"=USERID >> .env`

`echo "MINIO_KEY"=PWD >> .env`

### 3. Load credentials securely

In [3]:
load_dotenv() # add this line
user = os.getenv('MINIO_ACCESS')
key = os.getenv('MINIO_KEY')

In [4]:
user

'elan'

### 4. Initialize s3 client

In [5]:
import boto3
s3 = boto3.resource('s3',
                endpoint_url="http://minio-0:9000",
                aws_access_key_id=user,
                aws_secret_access_key=key)

### 5. Load data into workspace

In [68]:
# Print buckets in data server
buckets = s3.buckets.all()
for b,bucket in enumerate(buckets):
    print(b, "Bucket: " ,bucket.name, bucket.creation_date)

0 Bucket:  aish 2022-03-11 19:18:32.851000+00:00
1 Bucket:  boneagevalidationset 2022-03-17 14:51:10.405000+00:00
2 Bucket:  dummy 2022-03-16 22:17:54.173000+00:00
3 Bucket:  elan 2022-03-11 04:19:28.004000+00:00
4 Bucket:  ike 2022-03-14 15:12:48.441000+00:00
5 Bucket:  madi 2022-03-15 16:38:34.529000+00:00
6 Bucket:  neeraja 2022-03-11 19:20:49.062000+00:00
7 Bucket:  neerajanew 2022-03-11 19:29:13.226000+00:00
8 Bucket:  zach 2022-03-11 19:22:45.234000+00:00


In [27]:
def downloadDirectoryFroms3(s3_resource, bucketName, remoteDirectoryName):
    print('Attempting to download bucket, ', bucketName, ' folder, ', remoteDirectoryName)
    bucket = s3_resource.Bucket(bucketName)
    for obj in bucket.objects.filter(Prefix = remoteDirectoryName):
        dir_path = os.path.join(bucketName, os.path.dirname(obj.key))
        if not os.path.exists(dir_path):
            print('Creating dir: ', dir_path)
            os.makedirs(dir_path,exist_ok=True)
        filename = os.path.join(dir_path,os.path.basename(obj.key))
        bucket.download_file(obj.key, filename) # save to same path

In [38]:
# Download your bucket
bucketname = 'elan'
os.makedirs(bucketname,exist_ok=True)
# List objects information.
downloadDirectoryFroms3(s3,bucketname,'')

Attempting to download bucket,  elan  folder,  


## D. Upload data from workspace to bucket in MINIO using s3

### 1. Define function to delete bucket if it already exists

In [44]:
from botocore.client import ClientError

In [48]:
def delete_bucket(s3, bucketname):
    try:
        s3.meta.client.head_bucket(Bucket=bucketname)
        bucket = s3.Bucket(bucketname)
        response = bucket.objects.all().delete()
        print('Deleted bucket objects: \n',response)
        response = bucket.delete(
            ExpectedBucketOwner='string'
        )
        print('Deleted bucket: \n',response)
    except ClientError:
        print('Bucket does not exist')

### 2. Set new bucket and upload folder

In [7]:
new_bucket = "monailabel_datasets"
upload_folder = "/workspace/MONAILabel/data"

### 3. Delete bucket if it exists and create empty bucket - Only for test purposes - don't delete project data buckets!

In [50]:
delete_bucket(s3,new_bucket)

Bucket does not exist


### 4. Upload Data to bucket

In [8]:
def upload_files(s3,bucketname,folderpath):
    bucket = s3.Bucket(bucketname)
    if not bucket.creation_date:
        s3.create_bucket(Bucket=bucketname)

    for subdir, dirs, files in os.walk(folderpath):
        for file in files:
            full_path = os.path.join(subdir, file)
            with open(full_path, 'rb') as data:
                bucket.put_object(Key=full_path[len(folderpath)+1:], Body=data)

In [9]:
upload_files(s3,new_bucket,upload_folder)

print("Upload Done")

ClientError: An error occurred (InvalidBucketName) when calling the CreateBucket operation: The specified bucket is not valid.

### 5. Go to MINIO Console and check if data is uploaded

<img style="width: 40%" src="im/cap3.PNG">